
try to make this clever

llm scaffold?


In [ ]:

async function llmScaffold(github) {

  if(!github) {
    console.error('Provide a github repository for comparison and updating')
  }

  // TODO: feed the llm the proper project components as necessary to get it to generate code files
  

}

module.exports = llmScaffold




llm code?


In [ ]:




import path from "path"
import {getLlama, LlamaChatSession} from "node-llama-cpp"
import process from "process"

const HOMEPATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
//const __dirname = path.dirname(fileURLToPath(import.meta.url));

let llama
let model
let context
let session
let initialChatHistory


async function initSession(context2, prompt) {
  if(!context2) {
    context = await model.createContext()
  } else {
    context = context2
  }

  session = new LlamaChatSession({
      contextSequence: context.getSequence(),
      systemPrompt: prompt ? prompt : '```markdown\n# You are a\n##helpful coding module\nnamed Llama that responds to every request using beautifully crafted `markdown`. Return only a single code block in the specified language without reasoning or instructions if requested.\n</think>\n...```\n'
  })
  // initialize the model
  //console.log(await session.prompt())
  initialChatHistory = session.getChatHistory();

  // Reset the chat history
  session.setChatHistory(initialChatHistory);
  return session
}

async function createSession() {
  if(!llama) {
    llama = await getLlama();
  }
  if(!model) {
      model = await llama.loadModel({
          modelPath: path.join(HOMEPATH, "llama.cpp", "models", 'DeepSeek-R1-Distill-Llama-8B-Q6_K.gguf' ),
          //contextSize: 2048
      });
  }

  await initSession()

  return session
}

async function getSession() {
  if(!session) {
    await createSession()
  }
  return session
}

async function llmCode(prompt, session2) {
  if(!session2) {
    session2 = await getSession()
  }
  let result = await session2.prompt(prompt, {
    //maxTokens: context.contextSize,
    onTextChunk: function (text) {
      process.stdout.write(text)
    }
  })
  if(session == session2)
    session2.setChatHistory(initialChatHistory);
  return result
}

export default {
  createSession,
  initSession,
  getSession,
  llmCode,
}





bash project files?


In [ ]:

async function generateBash(github) {

  // TODO: local pull

  // TODO: generate bash

  // TODO: mount project directory

  // TODO: execute new docker file on project

}

module.exports = generateBash



generate code?


In [ ]:


async function generateCode(github) {
  
  // TODO: local pull

  // TODO: compare existing project files

  // TODO: create new files

  // TODO: populate code files with goals

  // TODO: populate goals with actual code

}

module.exports = generateCode

